In [3]:
import numpy as np
import pandas as pd
import torch
import os
os.environ ["KMP_DUPLICATE_LIB_OK"] ="TRUE"
from torch import nn
from d2l import torch as d2l
from torch.nn import functional as F
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.1635],
        [-0.1713]], grad_fn=<AddmmBackward0>)

In [4]:
print(net[2].state_dict()) # 第二层的参数

OrderedDict([('weight', tensor([[ 0.3066, -0.0282, -0.1767,  0.2233,  0.3512,  0.1433, -0.1984,  0.1899]])), ('bias', tensor([-0.3031]))])


In [5]:
# 访问底层数值
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)


<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.3031], requires_grad=True)
tensor([-0.3031])


In [6]:
net[2].weight.grad == None

True

一次性访问所有参数

In [7]:

print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [8]:
# 另一种访问网络参数的方式
net.state_dict()['2.bias'].data

tensor([-0.3031])

In [9]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net


rgnet = nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)

tensor([[-0.2617],
        [-0.2616]], grad_fn=<AddmmBackward0>)

In [11]:
# rgnet的结构
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [12]:
#访问第一个主要块中、第二个子块第一层的偏置项
rgnet[0][1][0].bias.data

tensor([-0.3296,  0.2795,  0.1189, -0.1656,  0.0411, -0.4062, -0.2349,  0.0854])

In [13]:
# 内置初始化器
def init_normal(m):
    if type(m) == nn.Linear:
        # 权重初始化为（0，0.01）的高斯分布
        nn.init.normal_(m.weight, mean = 0, std = 0.01)
        # 偏置全部初始化为0
        nn.init.zeros_(m.bias)
    

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]



(tensor([-0.0100, -0.0031, -0.0009,  0.0126]), tensor(0.))

In [14]:
# 参数初始化为给定的常数1
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [15]:
def init_xavier(m):
    if type(m) == nn.Linear:
        #将输入的权重张量用均匀分布进行初始化
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

# 不同层不同的初始化参数的方法
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.3083, -0.4515,  0.0248, -0.4088])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [16]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        # 绝对值小于5的进行减枝
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  0.0000, -8.3701, -0.0000],
        [ 0.0000, -0.0000,  0.0000, -9.8602]], grad_fn=<SliceBackward0>)